In [2]:
import numpy as np
import pandas as pd
from ape import accounts, chain, Contract, networks, project

In [3]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [4]:
head_block = chain.blocks.head.number

In [6]:
head_block

18399698

In [7]:
start = head_block - 7200 * 365 * 2  # last 2 years
stop = head_block
step = 2400  # sample price once per 8 hour

In [9]:
# relevant block numbers to query for
blocks = range(start, stop, step)
blocks

range(13143698, 18399698, 2400)

In [10]:
pool = Contract("0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640")  # USDC/ETH 0.05%

In [11]:
pool

<UniswapV3Pool 0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640>

In [12]:
print(f"token0: {pool.token0()}, token1: {pool.token1()}, fee: {pool.fee()}")

token0: 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48, token1: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2, fee: 500


In [13]:
fp = f"data/price_{pool.address}_{start}_{stop}_{step}.csv"
fp

'data/price_0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640_13143698_18399698_2400.csv'

In [19]:
is_head = True
for block in range(start, stop, step):
    # get the sqrt price data at block
    slot0 = pool.slot0(block_identifier=block)
    row = {'block_number': [block], 'sqrt_price_x96': [slot0.sqrtPriceX96]}
    print(f"Pool slot0.sqrtPriceX96 at block {block}: {slot0.sqrtPriceX96}", end='\r')

    # convert to pdf dataframe then append to file
    df = pd.DataFrame(data=row)
    df.to_csv(fp, mode='a', index=False, header=is_head)

    if is_head:
        is_head = False